In [ ]:
from ray.rllib.algorithms.ppo import PPOConfig

config = (  # 1. Configure the algorithm,
    PPOConfig()
    .environment("Taxi-v3")
    .env_runners(num_env_runners=2)
    .framework("torch")
    .training(model={"fcnet_hiddens": [64, 64]})
    .evaluation(evaluation_num_env_runners=1)
)

algo = config.build()  # 2. build the algorithm,

for _ in range(5):
    print(algo.train())  # 3. train it,

algo.evaluate()  # 4. and evaluate it

In [5]:
from pprint import pprint

from ray.rllib.algorithms.ppo import PPOConfig

config = (
    PPOConfig()
    .api_stack(
        enable_rl_module_and_learner=True,
        enable_env_runner_and_connector_v2=True,
    )
    .environment("CartPole-v1")
    .env_runners(num_env_runners=1)
)

algo = config.build()

for i in range(10):
    result = algo.train()
    result.pop("config")
    pprint(result)

    if i % 5 == 0:
        checkpoint_dir = algo.save()
        print(f"Checkpoint saved in directory {checkpoint_dir}")

/home/zhangzhibo/anaconda3/envs/mujoco/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:557: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/zhangzhibo/anaconda3/envs/mujoco/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/home/zhangzhibo/anaconda3/envs/mujoco/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning

{'date': '2024-09-11_14-57-15',
 'done': False,
 'env_runners': {'agent_episode_returns_mean': {'default_agent': 22.15},
                 'episode_duration_sec_mean': 0.02796674086013809,
                 'episode_len_max': 60,
                 'episode_len_mean': 22.15,
                 'episode_len_min': 9,
                 'episode_return_max': 60.0,
                 'episode_return_mean': 22.15,
                 'episode_return_min': 9.0,
                 'module_episode_returns_mean': {'default_policy': 22.15},
                 'num_agent_steps_sampled': {'default_agent': 4000},
                 'num_agent_steps_sampled_lifetime': {'default_agent': 4000},
                 'num_env_steps_sampled': 4000,
                 'num_env_steps_sampled_lifetime': 4000,
                 'num_episodes': 179,
                 'num_module_steps_sampled': {'default_policy': 4000},
                 'num_module_steps_sampled_lifetime': {'default_policy': 4000},
                 'sample': 5.43356093

In [6]:
from ray import train, tune

config = (
    PPOConfig()
    .api_stack(
        enable_rl_module_and_learner=True,
        enable_env_runner_and_connector_v2=True,
    )
    .environment("CartPole-v1")
    .training(
        lr=tune.grid_search([0.01, 0.001, 0.0001]),
    )
)

tuner = tune.Tuner(
    "PPO",
    param_space=config,
    run_config=train.RunConfig(
        stop={"env_runners/episode_return_mean": 150.0},
    ),
)

tuner.fit()

2024-09-11 15:24:01,018	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/zhangzhibo/ray_results/PPO_2024-09-11_15-21-05' in 0.0299s.
2024-09-11 15:24:01,031	INFO tune.py:1041 -- Total run time: 175.61 seconds (175.52 seconds for the tuning loop).


ResultGrid<[
  Result(
    metrics={'timers': {'env_runner_sampling_timer': 2.787463718417977, 'learner_update_timer': 7.211243212276247, 'synch_weights': 0.021947416356418725, 'synch_env_connectors': 0.013888692630123614}, 'env_runners': {'agent_episode_returns_mean': {'default_agent': 154.08}, 'num_agent_steps_sampled_lifetime': {'default_agent': 612000}, 'episode_return_min': 58.0, 'episode_return_max': 374.0, 'episode_len_min': 58, 'num_env_steps_sampled_lifetime': 1156000, 'episode_duration_sec_mean': 0.20654275377746673, 'num_module_steps_sampled': {'default_policy': 4000}, 'num_module_steps_sampled_lifetime': {'default_policy': 612000}, 'module_episode_returns_mean': {'default_policy': 154.08}, 'sample': 2.710674745660689, 'episode_len_mean': 154.08, 'episode_return_mean': 154.08, 'num_env_steps_sampled': 4000, 'num_episodes': 28, 'num_agent_steps_sampled': {'default_agent': 4000}, 'episode_len_max': 374, 'time_between_sampling': 7.495257776348786}, 'num_agent_steps_sampled_life

In [7]:
from ray import train, tune

# Tuner.fit() allows setting a custom log directory (other than ~/ray-results).
tuner = tune.Tuner(
    "PPO",
    param_space=config,
    run_config=train.RunConfig(
        stop={"num_env_steps_sampled_lifetime": 20000},
        checkpoint_config=train.CheckpointConfig(checkpoint_at_end=True),
    ),
)

results = tuner.fit()

# Get the best result based on a particular metric.
best_result = results.get_best_result(
    metric="env_runners/episode_return_mean", mode="max"
)

# Get the best checkpoint corresponding to the best result.
best_checkpoint = best_result.checkpoint

2024-09-11 15:42:32,493	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/home/zhangzhibo/ray_results/PPO_2024-09-11_15-41-25' in 0.0669s.
2024-09-11 15:42:33,061	INFO tune.py:1041 -- Total run time: 67.38 seconds (66.71 seconds for the tuning loop).


In [8]:
print(f"Best learning rate: {best_result.config['lr']}")

Best learning rate: 0.001
